In [15]:
import torch
import math
import numpy as np
from matplotlib import pyplot as plt
import os
import sys
sys.path.append('..')
import ROOT
from Tools import syncer 
from Tools import user
from Tools import helpers
import itertools

from PIL import Image
import glob

In [2]:
class NN:
    def __init__(self,nfeatures,coefficient_names):
        self.nfeatures         = nfeatures
        self.coefficient_names = coefficient_names
        self.combination_list=list(itertools.chain.from_iterable(itertools.combinations_with_replacement(self.coefficient_names, i) for i in np.arange(0,3)))
        self.n_hat = {combination: self.make_NN() for combination in self.combination_list}
        
    def make_NN(self, hidden_layers  = [32, 32, 32, 32]):
        '''
        Creates the Neural Network Architecture
        '''
        model_nn = [torch.nn.BatchNorm1d(self.nfeatures), torch.nn.ReLU(), torch.nn.Linear(self.nfeatures, hidden_layers[0])]
        for i_layer, layer in enumerate(hidden_layers):
            model_nn.append(torch.nn.Linear(hidden_layers[i_layer], hidden_layers[i_layer+1] if i_layer+1<len(hidden_layers) else 1))
            if i_layer+1<len(hidden_layers):
                model_nn.append( torch.nn.ReLU() )
        return torch.nn.Sequential(*model_nn)

    def evaluate_NN(self, features):
        '''Evaluate Neural Network: The zeroth dimension of features is the number of data points and and the first dimension
        is the number of features(variables). Returns the output of the NNs of dimensions: (noutput,ndatapoints)
        '''
        noutputs=len(self.combination_list)
        ndatapoints=features.shape[0]
        
        output=torch.zeros((noutputs,ndatapoints))
        for i in range(noutputs):
            x=self.n_hat[self.combination_list[i]](features)
            if i==0:
                output[i,:]=1
            else:
                output[i,:]=torch.flatten(x)            
        return output

    def predict_r_hat2(self, predictions,eft):
        return torch.add( 
        torch.sum( torch.stack( [(1. + predictions[(c,)]*eft[c])**2 for c in coefficients ]), dim=0),
        torch.sum( torch.stack( [torch.sum( torch.stack( [ predictions[(c_1,c_2)]*eft[c_2] for c_2 in coefficients[i_c_1:] ]), dim=0)**2 for i_c_1, c_1 in enumerate(coefficients) ] ), dim=0 ) )   

    def save(self,fileName):
        outfile = open(fileName,'wb')
        pickle.dump(self, outfile)
        outfile.close()
        
    @classmethod
    def load(self, fileName):
        infile = open(fileName,'rb')
        print(fileName)
        new_dict = pickle.load(infile)
        infile.close()
        return new_dict

In [3]:
def make_weight_ratio(weights, **kwargs):
    eft      = kwargs
    result = torch.ones(len(weights[()])) 
    for combination in combinations:
        if len(combination)==1:
            result += eft[combination[0]]*weights[combination]/weights[()]
        elif len(combination)==2:# add up without the factor 1/2 because off diagonals are only summed in upper triangle 
            result += (0.5 if len(set(combination))==1 else 1.)*eft[combination[0]]*eft[combination[1]]*weights[combination]/weights[()]
    return result

In [4]:
# loss functional
def f_loss(r_NN, features, predictions, base_points):
    loss = -0.5*weights[()].sum()
    for i_base_point, base_point in enumerate(base_points):
        fhat  = 1./(1. + r_NN.predict_r_hat2(predictions, base_point))
        loss += ( torch.tensor(weights[()])*( -0.25 + base_point_weight_ratios[i_base_point]*fhat**2 + (1-fhat)**2 ) ).sum()
    return loss

In [5]:
plot_directory="13_7_2022"
nEvents=30000
learning_rate = 1e-3
device        = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epoch       = 3000
plot_every    = 100



In [6]:
# training data
import ZH_Nakamura

ZH_Nakamura.feature_names = ZH_Nakamura.feature_names[0:6] # restrict features
features   = ZH_Nakamura.getEvents(nEvents)[:,0:6]
feature_names  = ZH_Nakamura.feature_names
plot_options   = ZH_Nakamura.plot_options
plot_vars      = ZH_Nakamura.feature_names

mask       = (features[:,feature_names.index('pT')]<900) & (features[:,feature_names.index('sqrt_s_hat')]<1800) 
features = features[mask]

nfeatures = len(features[0]) 
weights    = ZH_Nakamura.getWeights(features, ZH_Nakamura.make_eft())


#pT=features[:,feature_names.index('pT')]



coefficients   =  ( 'cHW', 'cHWtil') 
combinations   =  [(), ('cHW',), ('cHWtil',), ('cHW','cHW'), ('cHW','cHWtil'),('cHWtil','cHWtil')]
combinations2 = [(), ('cHW',), ('cHWtil',), ('cHW','cHW'), ('cHW','cHWtil'),('cHWtil','cHW'),('cHWtil','cHWtil')]
base_points = [ {'cHW':value1, 'cHWtil':value2} for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5]]

rows, columns = np.triu_indices(len(coefficients)+1)
w=np.zeros((len(coefficients)+1,len(coefficients)+1,features.shape[0]))

for i in range(0,len(combinations)):
    w[rows[i],columns[i],:]=torch.from_numpy(weights[combinations[i]]).float().to(device)
    if rows[i]!=columns[i]:
        w[columns[i],rows[i],:]=w[rows[i],columns[i],:]

features = torch.from_numpy(features).float().to(device)


Requested 30000 events. Simulated 30000 events and 30000 survive pT_min cut of 0.


In [7]:
coefficients = tuple(filter( lambda coeff: coeff in coefficients, list(coefficients))) 
combinations = tuple(filter( lambda comb: all([c in coefficients for c in comb]), combinations)) 

base_points    = list(map( lambda b:ZH_Nakamura.make_eft(**b), base_points ))

In [8]:
r_NN=NN(nfeatures,coefficients)

In [9]:
base_point_weight_ratios = list( map( lambda base_point: make_weight_ratio( weights, **base_point ), base_points ) )

In [10]:
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(sum([list(model.parameters()) for model in r_NN.n_hat.values()],[]), lr=learning_rate)

losses = []

tex = ROOT.TLatex()
tex.SetNDC()
tex.SetTextSize(0.04)
hist_colors=['b','g', 'r', 'c', 'm','y']

In [11]:
for nn in r_NN.n_hat.values():
    nn.train()

for epoch in range(n_epoch):
    #print("epoch: ", epoch)
    # Forward pass: compute predicted y by passing x to the model.
    predictions = {combination:r_NN.n_hat[combination](features).squeeze() for combination in combinations}
    
    #rows, columns = np.triu_indices(len(coefficients)+1)
    w_predicted=np.zeros((len(coefficients)+1, len(coefficients)+1, features.shape[0]))      
    for i in range(1,len(combinations)):
        w_predicted[rows[i],columns[i],:]=predictions[combinations[i]].squeeze().cpu().detach().numpy()
        if rows[i]!=columns[i]:
            w_predicted[columns[i],rows[i],:]=w_predicted[rows[i],columns[i],:]
    
    wp=np.zeros((len(coefficients),features.shape[0]))
    for i in range(0,len(coefficients)):
        wp[i,:]=2*np.sum(w_predicted[:,i+1,:],0)
    
    wpp=np.zeros((len(coefficients),len(coefficients),len(coefficients),features.shape[0]))
    for i in range(0,len(coefficients)):
        for l in range(0,len(coefficients)):
            for k in range(0,len(coefficients)):
                wpp[i,l,k,:]=2*w_predicted[i,l+1,:]*w_predicted[i,k+1,:]
    
    wpp=np.sum(wpp,0)
               
    # Compute and print loss.
    loss = f_loss(r_NN,features, predictions,base_points)
    losses.append(loss.item())
    if epoch % 100 == 99:
        print("epoch", epoch, "loss",  loss.item())

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    
    

    if (epoch % plot_every)==0:
        with torch.no_grad():
                
            #print("epoch", epoch, "loss inside loop",  loss_train.item())
            #print("epoch", epoch, "test loss",  loss_test.item())
            
            for var in ['pT']:
                binning     = plot_options[var]['binning']
                np_binning= np.linspace(binning[1], binning[2], 1+binning[0])
                
                # Linear Terms
                hist_truth_0, bins  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w[0,0,:])
                bins = bins[1:]

               
                
                
                for i in range(1,len(coefficients)+1):
                    #plt.hist(features[:,feature_names.index(var)], bins, histtype='step', label='Yield',weights=w[0,0,:],color='black')
                    plt.hist(features[:,feature_names.index(var)], bins, histtype='step', label='truth',weights=w[0,i,:],linestyle=('dashed'), color=hist_colors[i])
                    plt.hist(features[:,feature_names.index(var)], bins, histtype='step', label='predicted',weights=w[0,0,:]*wp[i-1,:], color = hist_colors[i])
                    plt.legend(['Yield ' + coefficients[i-1], 'Truth '+ coefficients[i-1], 'Predicted '+ coefficients[i-1]])
                    plt.legend(['Truth '+ coefficients[i-1], 'Predicted '+ coefficients[i-1]])
                    plt.savefig(os.path.join(plot_directory, "_epoch_%05i_%s__"%(epoch, var)+coefficients[i-1] + ".png" ))
                    plt.close()
                
                
                a = np.arange(len(coefficients)**2).reshape(len(coefficients),len(coefficients))
                for l in range(0,len(coefficients)):
                    for k in range(0,len(coefficients)):
                        label=combinations2[a[l,k]+len(coefficients)+1][0]+ '_' + combinations2[a[l,k]+len(coefficients)+1][1]
                        #plt.hist(features[:,feature_names.index(var)], bins, histtype='step', label='Yield',weights=w[0,0,:],color='black')
                        plt.hist(features[:,feature_names.index(var)], bins, histtype='step', label='truth',weights=w[l+1,k+1,:],linestyle=('dashed'), color=hist_colors[a[l,k]+len(coefficients)])
                        plt.hist(features[:,feature_names.index(var)], bins, histtype='step', label='predicted',weights=w[0,0,:]*wpp[l,k,:], color = hist_colors[a[l,k]+len(coefficients)])
                        #plt.legend(['Yield ' + label, 'Truth '+ label, 'Predicted '+ label])
                        plt.legend(['Truth '+ label, 'Predicted '+ label])
                        plt.savefig(os.path.join(plot_directory, "_epoch_%05i_%s_"%(epoch, var)+label+".png"))
                        plt.close()
                    
                
               

epoch 99 loss 2.577893692619988
epoch 199 loss 2.566674588236542
epoch 299 loss 2.5505626438403124
epoch 399 loss 2.528638642200234
epoch 499 loss 2.5106022076374885
epoch 599 loss 2.5051012142753883
epoch 699 loss 2.503134481915266
epoch 799 loss 2.5021203458470658
epoch 899 loss 2.4991661541704198
epoch 999 loss 2.4958345277577307
epoch 1099 loss 2.495527082833928
epoch 1199 loss 2.495366250644571
epoch 1299 loss 2.4952361218816814
epoch 1399 loss 2.495107610877079
epoch 1499 loss 2.494428427622089
epoch 1599 loss 2.493840135930059
epoch 1699 loss 2.493532989961228
epoch 1799 loss 2.4933820018023987
epoch 1899 loss 2.4931926864943197
epoch 1999 loss 2.4928787140766877
epoch 2099 loss 2.4925531581717246
epoch 2199 loss 2.492238707806419
epoch 2299 loss 2.49202000252544
epoch 2399 loss 2.4917508559911994
epoch 2499 loss 2.491469412558504
epoch 2599 loss 2.4911365995000625
epoch 2699 loss 2.4909434316325076
epoch 2799 loss 2.4903281493845286
epoch 2899 loss 2.4901061011425667
epoch 2999

In [18]:
for i in range(1,len(coefficients)+1):
    pTFiles=[]
    for file in os.listdir(os.getcwd()+'/13_7_2022'):
        # check only text files
        string="__" + coefficients[i-1] + ".png"
        if file.endswith(string):
            pTFiles.append(file)
            #print(file)
    frames=[]
    for image in pTFiles:
        new_frame = Image.open(os.getcwd()+'/13_7_2022/'+image)
        frames.append(new_frame)

    frames[0].save('13_7_2022__'+ coefficients[i-1]+'.gif', format='GIF',
                   append_images=frames[1:],
                   save_all=True,
                   duration=200, loop=0)

_epoch_00000_pT__cHW.png
_epoch_00100_pT__cHW.png
_epoch_00200_pT__cHW.png
_epoch_00300_pT__cHW.png
_epoch_00400_pT__cHW.png
_epoch_00500_pT__cHW.png
_epoch_00600_pT__cHW.png
_epoch_00700_pT__cHW.png
_epoch_00800_pT__cHW.png
_epoch_00900_pT__cHW.png
_epoch_01000_pT__cHW.png
_epoch_01100_pT__cHW.png
_epoch_01200_pT__cHW.png
_epoch_01300_pT__cHW.png
_epoch_01400_pT__cHW.png
_epoch_01500_pT__cHW.png
_epoch_01600_pT__cHW.png
_epoch_01700_pT__cHW.png
_epoch_01800_pT__cHW.png
_epoch_01900_pT__cHW.png
_epoch_02000_pT__cHW.png
_epoch_02100_pT__cHW.png
_epoch_02200_pT__cHW.png
_epoch_02300_pT__cHW.png
_epoch_02400_pT__cHW.png
_epoch_02500_pT__cHW.png
_epoch_02600_pT__cHW.png
_epoch_02700_pT__cHW.png
_epoch_02800_pT__cHW.png
_epoch_02900_pT__cHW.png
_epoch_00000_pT__cHWtil.png
_epoch_00100_pT__cHWtil.png
_epoch_00200_pT__cHWtil.png
_epoch_00300_pT__cHWtil.png
_epoch_00400_pT__cHWtil.png
_epoch_00500_pT__cHWtil.png
_epoch_00600_pT__cHWtil.png
_epoch_00700_pT__cHWtil.png
_epoch_00800_pT__cHWtil.pn

In [19]:
for l in range(0,len(coefficients)):
    for k in range(0,len(coefficients)):
        label=combinations2[a[l,k]+len(coefficients)+1][0]+ '_' + combinations2[a[l,k]+len(coefficients)+1][1]
        pTFiles=[]
        for file in os.listdir(os.getcwd()+'/13_7_2022'):
            # check only text files
            string="_"+label+".png"
            if file.endswith(string):
                pTFiles.append(file)
                #print(file)
        frames=[]
        for image in pTFiles:
            new_frame = Image.open(os.getcwd()+'/13_7_2022/'+image)
            frames.append(new_frame)

        frames[0].save('13_7_2022__'+ label+'.gif', format='GIF',
                       append_images=frames[1:],
                       save_all=True,
                       duration=200, loop=0)